# This file is to finish the udacity nano degree problem for Anna_KaRNNa with Pytorch

This file is to build a chracter-wise RNN trained on Anna Kerenia.

## 1. Load Packages

In [8]:
import time
from collections import namedtuple
import numpy as np
import torch

## 2. Load Data

In [14]:
with open('anna.txt','r') as f:
    text = f.read()
vocab =sorted(set(text))
vocab_to_int = {c: i for i,c in enumerate(vocab)}
int_to_vocab =dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text])

One of the best first line of a book ever.

In [54]:
lr = 0.001

In [55]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

## 3. Make training mini-batches

In [26]:
# Follow the udacity's books function to create the batch-iterator here

In [29]:
def get_batches(arr, batch_size, n_steps):
    ''' Create generator that returns the batches of size 
        batch_size * n_steps
        Here we follow the udacity's notebook's rule, we will remove the last piece
        of the samples smaller than the batch_size.
    '''
    size = batch_size * n_steps
    n_batches = len(arr)/size
    
    # Here we only keep enough numbers 
    arr = arr[:size*n_batches]
    
    # reshape batches into batch_size row
    arr = arr.reshape((batch_size,-1))
    
    for n in range(0,arr.shape[1],n_steps):
        x = arr[:,n:n+n_steps]
        y_temp = arr[:,n+1:n+n_steps+1]
        
        # for the last batch, the y will be short with one.
        # we will pad the last value with 0
        
        y = np.zeros(x.shape)
        y[:,:y_temp.shape[1]]=y_temp
        
        yield x,y

In [30]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [32]:
print('x \n', x[:10, :10])
print('\n y \n', y[:10, :10])

('x \n', array([[31, 64, 57, 72, 76, 61, 74,  1, 16,  0],
       [ 1, 57, 69,  1, 70, 71, 76,  1, 63, 71],
       [78, 65, 70, 13,  0,  0,  3, 53, 61, 75],
       [70,  1, 60, 77, 74, 65, 70, 63,  1, 64],
       [ 1, 65, 76,  1, 65, 75, 11,  1, 75, 65],
       [ 1, 37, 76,  1, 79, 57, 75,  0, 71, 70],
       [64, 61, 70,  1, 59, 71, 69, 61,  1, 62],
       [26,  1, 58, 77, 76,  1, 70, 71, 79,  1],
       [76,  1, 65, 75, 70,  7, 76, 13,  1, 48],
       [ 1, 75, 57, 65, 60,  1, 76, 71,  1, 64]]))
('\n y \n', array([[64., 57., 72., 76., 61., 74.,  1., 16.,  0.,  0.],
       [57., 69.,  1., 70., 71., 76.,  1., 63., 71., 65.],
       [65., 70., 13.,  0.,  0.,  3., 53., 61., 75., 11.],
       [ 1., 60., 77., 74., 65., 70., 63.,  1., 64., 65.],
       [65., 76.,  1., 65., 75., 11.,  1., 75., 65., 74.],
       [37., 76.,  1., 79., 57., 75.,  0., 71., 70., 68.],
       [61., 70.,  1., 59., 71., 69., 61.,  1., 62., 71.],
       [ 1., 58., 77., 76.,  1., 70., 71., 79.,  1., 75.],
       [ 1., 65

## 4. Model Building

In [33]:
# Here we build the LSTM neural network models.

In [51]:
## Need to rewrite this function

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class LSTM_Char(nn.Module):
    
    def __init__(self,hidden_dim,n_steps,n_layers,label_size,batch_size,drop_out=0.5,lr=0.001):
        
        super(LSTM_Char,self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.label_size = label_size
        self.n_layers = n_layers
        self.lstm = nn.LSTM(input_size= label_size ,hidden_size = hidden_dim,num_layers = n_layers,\
                            dropout=drop_out,batch_first=True)
        self.hidden = self.init_hidden(n_steps)
        self.out = nn.Linear(hidden_dim,label_size)
        self.init_weight()
        
    def init_hidden(self,n_seqs):
        ## Two layer of hidden, first is for the input x, second is for the hidden state
        weight = next(self.parameters()).data
        return (Variable(weight.new(self.n_layers,n_seqs,self.hidden_dim).zero_()),
                Variable(weight.new(self.n_layers,n_seqs,self.hidden_dim).zero_()))
    
    def forward(self,x,hc):
        # Here we put the hc in the tuple
        x,(h,c) = self.lstm(x,hc)
        x = self.dropout(x)
        
        # Put all the LSTM outputs together
        x = x.view(x.size()[0]*x.size()[1],self.n_hidden)
        
        return x,(h,c)
    
    def predict(self,char,h=None,cuda=False,top_k=None):
        ''' Given a character, predict the next character.
            Return the predicted character and the hidden state.
        '''
        # Currently we don't consider cuda here.
        if h is None:
            h=self.init_hidden(1)
        x = np.array(vocab_to_int[char])
        x = one_hot_encode(x,self.label_size)
        
        inputs = Variable(torch.from_numpy(x),volatile = True)
        
        h = tuple(Variable(each.data,volatile = True) for each in h)
        
        out,h= self.forward(inputs,h)
        
        p = F.softmax(out).data
        
        if top_k is None:
            top_ch = np.arange(self.label_size)
        else:
            p,top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        p = p.numpy().squeeze()
        
        char = np.random.choice(top_ch, p=p/p.sum())
        
        return int_to_vocab[char],h
    
    def init_weight(self):
        initrange = 0.1
        #set bias tensor to all zeros
        self.out.bias.data.fill_(0)
        # set weight to randeom uniform
        self.out.weight.data.uniform_(-1,1)
        

## 5. Model Training

In [52]:
net = LSTM_Char(hidden_dim = 512,n_steps=100,\
                n_layers = 2,label_size = len(vocab),\
                batch_size=128)

In [56]:
net.train()
opt = torch.optim.Adam(net.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss()

In [59]:
# Here we can do the split between training and test data
val_frac = 0.2
val_index = int(len(encoded)*(1-val_frac))
data,val_data = encoded[:val_index],encoded[val_index:]

## 6. Check Results

In [ ]:
clip = 5
counter = 0
for e in range(epoch):
    h = net.init_hidden(n_seqs)
    for x, y in get_batches(data,n_seqs,n_steps):
        counter += 1
        
        # Need to do one-hot encoding and make them the torch tensors
        x = one_hot_encode(x,label_size)
        x, y =torch.from_numpy(x),torch.from_numpy(y)
        
        inputs,targetss = Variable(x), Variable(y)
        
        # Create new variables for the hidden state, otherwise we'd backprop
        # thourgh the entire training history
        ## Need to double check which options works better, detach or create new variables
        
        h = tuple([Variable(each.data) for each in h])
        
        net.zero_grad()
        
        output, h = net.forward(inputs,h)
        
        loss = criterion(output,targets.view(n_seqs*n_steps))
        
        loss.backward()
        
        # clip_grad_norm helps prevent the exploding gradient
        # double check this
        nn.utils.clip_grad_norm(net.parameters,clip)
        
        opt.step()
        
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(n_seqs)
            val_losses = []
            for x, y in get_batches(val_data,n_seqs,n_steps):
                x = one_hot_encode(x,label_size)
                x,y = torch.from_numpy(x),torch.from_numpy(y)
                
                val_h = tuple([Variable(each.data,volatile= True) for each in val_h ])
                
                inputs, targets = Variable(x,volatile=True),Variable(y,volatile=True)
                
                output,val_h = net.forward(inputs,val_h)
                
                val_loss = criterion(output,targets.view(n_seqs*n_steps))
                
                val_losses.append(val_loss.data[0])
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.data[0]),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
            